In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [ ]:
# Set up the data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_labels = pd.read_csv('Emergency_Vehicles/train.csv')
# Convert the column to strings
train_labels['emergency_or_not'] = train_labels['emergency_or_not'].astype(str)
train_generator = train_datagen.flow_from_dataframe(
    train_labels,
    directory='Emergency_Vehicles/train',
    x_col='image_names',
    y_col='emergency_or_not',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')


Found 1380 validated image filenames belonging to 2 classes.


/home/u184510/.local/lib/python3.9/site-packages/keras/preprocessing/image.py:1137: UserWarning: Found 266 invalid image filename(s) in x_col="image_names". These filename(s) will be ignored.
  warnings.warn(


In [6]:
# Load the pre-trained DenseNet-169 model and freeze the base layers
base_model = tf.keras.applications.DenseNet169(
    include_top=False, weights='imagenet', input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

# Add new fully connected layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Compile the model with binary cross-entropy loss and Adam optimizer
model = Model(inputs=base_model.input, outputs=predictions)

#freeze Layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20)

2023-03-17 06:31:28.504951: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /glob/development-tools/versions/oneapi/2023.0.1/oneapi/vpl/2023.0.0/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/tbb/2021.8.0/env/../lib/intel64/gcc4.8:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/rkcommon/1.10.0/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/ospray_studio/0.11.1/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/ospray/2.10.0/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/openvkl/1.3.1/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/oidn/1.4.3/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/mpi/2021.8.0//libfabric/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/mpi/2021.8.0//lib/release:/glob/development-tools/versions/oneapi

Epoch 1/20
43/43 [==============================] - 72s 1s/step - loss: 0.2923 - accuracy: 0.8761
Epoch 2/20
43/43 [==============================] - 55s 1s/step - loss: 0.1718 - accuracy: 0.9318
Epoch 3/20
43/43 [==============================] - 55s 1s/step - loss: 0.1262 - accuracy: 0.9547
Epoch 4/20
43/43 [==============================] - 54s 1s/step - loss: 0.1246 - accuracy: 0.9622
Epoch 5/20
43/43 [==============================] - 54s 1s/step - loss: 0.1145 - accuracy: 0.9607
Epoch 6/20
43/43 [==============================] - 53s 1s/step - loss: 0.0980 - accuracy: 0.9711
Epoch 7/20
43/43 [==============================] - 54s 1s/step - loss: 0.0851 - accuracy: 0.9763
Epoch 8/20
43/43 [==============================] - 54s 1s/step - loss: 0.0890 - accuracy: 0.9696
Epoch 9/20
43/43 [==============================] - 54s 1s/step - loss: 0.0911 - accuracy: 0.9718
Epoch 10/20
43/43 [==============================] - 54s 1s/step - loss: 0.0723 - accuracy: 0.9785
Epoch 11/20
43/43 [

In [7]:
# Load the test CSV file with filenames
t_test_df = pd.read_csv("Emergency_Vehicles/train.csv")

In [8]:
# Create a generator for the test images
t_test_datagen = ImageDataGenerator(rescale=1./255)
t_test_generator = t_test_datagen.flow_from_dataframe(
    t_test_df,
    directory="Emergency_Vehicles/train",
    x_col='image_names',
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False)

# Use the trained model to make predictions on the test data
preds = model.predict_generator(t_test_generator, steps=None)
# Remove any missing filenames from the test CSV file
missing_filenames = set(t_test_generator.filenames) - set(t_test_df['image_names'])
if missing_filenames:
    print(f'Removing {len(missing_filenames)} missing filenames from the test CSV file.')
    t_test_df = t_test_df[~t_test_df['image_names'].isin(missing_filenames)]

Found 1380 validated image filenames.


/home/u184510/.local/lib/python3.9/site-packages/keras/preprocessing/image.py:1137: UserWarning: Found 266 invalid image filename(s) in x_col="image_names". These filename(s) will be ignored.
  warnings.warn(
/home/u184510/tmp/ipykernel_470999/1260252469.py:14: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  preds = model.predict_generator(t_test_generator, steps=None)


In [9]:
li = []
for i in preds.ravel():
    if i>0.5:
        li.append(1)
    else:
        li.append(0)
     
# Create a DataFrame with the filenames and predictions
df = pd.DataFrame({'image_names': t_test_generator.filenames[:len(preds)], 'emergency_or_not': li})

test_inst_df=pd.merge(df, t_test_df, on='image_names', how='inner')

In [10]:
accuracy_score(df["emergency_or_not"],test_inst_df['emergency_or_not_y']) 

0.9855072463768116

In [11]:
confusion_matrix(df["emergency_or_not"],test_inst_df['emergency_or_not_y']) 

array([[723,  19],
       [  1, 637]])

In [12]:
# Save the model
model.save('models/densenet.h5')